<h1 style="color:red" align="center">Neural Networks : The basics made for all</h1>
<p align="center">By Bastien Lhopitallier and Robin Sobczyk</p>

<h2 style="color:green" align="center">Course 2 : [title to find]</h2>
course 2 summary